In [ ]:
import numpy as np
import torch
import torch.nn as nn


In [ ]:
trainData = np.loadtxt("bachelor-thesis/dataset/ECG5000/ECG5000_TRAIN.txt")
testData = np.loadtxt("bachelor-thesis/dataset/ECG5000/ECG5000_TEST.txt")

In [ ]:
train_labels = trainData[:, 0].astype(int)
train_signals = trainData[:, 1:]

test_labels = testData[:, 0].astype(int)
test_signals = testData[:, 1:]


In [ ]:
normal_train = train_signals[train_labels == 1]
normal_test  = test_signals[test_labels == 1]
abnormal_test = test_signals[test_labels != 1]


In [ ]:
x_train = torch.tensor(normal_train, dtype=torch.float32)
x_test_normal = torch.tensor(normal_test, dtype=torch.float32)
x_test_abnormal = torch.tensor(abnormal_test, dtype=torch.float32)
print(x_train.shape)
print(x_test_normal.shape)
print(x_test_abnormal.shape)



Defining the model

In [ ]:
class ECGTransformerAutoencoder(nn.Module):
    def __init__(
        self,
        signal_length=140,
        patch_size=10,
        d_model=64,
        nhead=4,
        num_layers=2
    ):
        super().__init__()

        self.signal_length = signal_length
        self.patch_size = patch_size
        self.d_model = d_model

        self.num_patches = signal_length // patch_size

        # Patch embedding
        self.patch_embed = nn.Linear(patch_size, d_model)

        # Learned positional embedding
        self.pos_embed = nn.Parameter(
            torch.zeros(1, self.num_patches, d_model)
        )

        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=128,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )

        # Decoder
        self.decoder = nn.Linear(d_model, patch_size)

    def forward(self, x):
        """
        x: [B, 140]
        """
        B, T = x.shape
        N = self.num_patches

        # 1) split into patches
        x = x.view(B, N, self.patch_size)          # [B, 14, 10]

        # 2) patch embedding
        x = self.patch_embed(x)                    # [B, 14, 64]

        # 3) add positional encoding
        x = x + self.pos_embed                     # [B, 14, 64]

        # 4) transformer encoder
        x = self.encoder(x)                        # [B, 14, 64]

        # 5) decode patches
        x = self.decoder(x)                        # [B, 14, 10]

        # 6) reconstruct signal
        x = x.contiguous().view(B, T)              # [B, 140]

        return x


In [ ]:
model = ECGTransformerAutoencoder()

with torch.no_grad():
    recon = model(x_train[:8])

print("Input shape:", x_train[:8].shape)
print("Reconstruction shape:", recon.shape)


In [ ]:
model = ECGTransformerAutoencoder()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
num_epochs = 30
batch_size = 64

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for i in range(0, len(x_train), batch_size):
        batch = x_train[i:i + batch_size]

        optimizer.zero_grad()

        recon = model(batch)
        loss = criterion(recon, batch)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= (len(x_train) // batch_size)

    print(f"Epoch {epoch+1:02d} | Train Loss: {train_loss:.4f}")
